In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

# restore saved variables
%store -r summary_general2_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_two_year'].values

In [4]:
#### CART
depth = [7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.5]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, 
                          learning_rate=learning_rate, 
                          depth=depth, 
                          estimators=estimators, 
                          seed=816)

In [6]:
cart_summary['FL_score'], ebm_summary['FL_score']

(0.5832411793596223, 0.5895942584641709)

### Lasso Stumps

In [7]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_two_year'].values

## columns 
cols = KY_X.columns

In [8]:
c_grid={'C': [0.002, 0.003, 0.004]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary

{'best_auc': 0.7389490385496335,
 'best_params': {'C': 0.004},
 'auc_diffs': 0.0002560205675981031,
 'FL_score': 0.5980977729896438}

### RiskSLIM

In [10]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_two_year'].values

## columns 
cols = KY_X.columns

In [17]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.001, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

17

In [18]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [19]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general2', 
                                max_coef = 5, 
                                max_coef_number = 15, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/21/19 @ 11:59 AM | 323 rows in lookup table
10/21/19 @ 11:59 AM | ------------------------------------------------------------
10/21/19 @ 11:59 AM | runnning initialization procedure
10/21/19 @ 11:59 AM | ------------------------------------------------------------
10/21/19 @ 11:59 AM | CPA produced 2 cuts
10/21/19 @ 11:59 AM | running naive rounding on 250 solutions
10/21/19 @ 11:59 AM | best objective value: 0.5290
10/21/19 @ 11:59 AM | rounding produced 1 integer solutions
10/21/19 @ 11:59 AM | best objective value is 0.6036
10/21/19 @ 11:59 AM | running sequential rounding on 250 solutions
10/21/19 @ 11:59 AM | best objective value: 0.5290
10/21/19 @ 11:59 AM | sequential rounding produced 3 integer solutions
10/21/19 @ 11:59 AM | best objective value: 0.5481
10/21/19 @ 11:59 AM | polishing 4 solutions
10/21/19 @ 11:59 AM | best objective value: 0.5481
10/21/19 @ 11:59 AM | polishing produced 4 integer solutions
10/21/19

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 11:59 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5481.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:16 PM | adding 238 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5491.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:37 PM | adding 242 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5500.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 12:54 PM | adding 242 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5499.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 01:11 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5527.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 01:28 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5495.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

In [20]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_train_score'])

(0.579126200774199, 0.7059693902927714)

In [21]:
#### save results
summary_general2_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary}
%store summary_general2_ky_inter_model

Stored 'summary_general2_ky_inter_model' (dict)


In [23]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["", "CART", "EBM", "Lasso Stumps", "RiskSLIM"],
           ["General",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Two Year KY_inter_model.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)